# Programa Ingenias+ Data Science

## Objetivo del proyecto

Predecir si una persona tendra un IMC anormal o no, en base a sus habitos y/o condiciones socioeconomicas

## Objetivo del notebook

Este notebook tiene como objetivo probar diferentes algoritmos de clasificacion para determinar cual puede brindarnos mejores parametros


## Importacion archivos y librerias

In [1]:
# imports de librerias y funciones
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,  OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

from utils.codificaciones import *
from utils.funcions import *

In [2]:
# importo mi csv
friesgo = pd.read_csv('datasets/friesgo.csv')

# Preprocesado

Codificando las variables categoricas y eliminando los nan

In [3]:
# Aqui eliminaremos los registro que tengan nulos en la variable objetivo, comenzando por reemplazar lo valores 99 : ns/nc
friesgo['mantiene_ha_alta'] = friesgo['mantiene_ha_alta'].replace(99, np.nan)
friesgo.mantiene_ha_alta.unique()

array([ 2.,  1., nan])

In [4]:
# Primero busco cuales columnas efectivamente son categoricas y cuales numericas
variables_categoricas = ['cod_provincia','tamanio_aglomerado','sexo','condicion_actividad','ansiedad_depresion'
                           ,'nivel_actividad_fisica','condicion_fumador','consumo_sal'
                           ,'colesterol_elevado','freq_cons_alc','es_diabetico','mantiene_ha_alta']
variables_numericas = ['edad','media_fv_diaria','imc_numerico']

In [7]:
# Creo una bd codificada a pesar de que ya esten codificados (para probar si algo cambia)
friesgo_cod = friesgo.copy()

In [8]:
calcular_nulos_y_porcentaje(friesgo_cod)

,Nulos,Porcentaje
id,0,0.0%
cod_provincia,0,0.0%
tamanio_aglomerado,0,0.0%
sexo,0,0.0%
edad,0,0.0%
condicion_actividad,0,0.0%
ansiedad_depresion,0,0.0%
nivel_actividad_fisica,254,0.87%
condicion_fumador,0,0.0%
mantiene_ha_alta,2235,7.65%


In [9]:
friesgo_cod = friesgo_cod.dropna()

In [10]:
calcular_nulos_y_porcentaje(friesgo_cod)

,Nulos,Porcentaje
id,0,0.0%
cod_provincia,0,0.0%
tamanio_aglomerado,0,0.0%
sexo,0,0.0%
edad,0,0.0%
condicion_actividad,0,0.0%
ansiedad_depresion,0,0.0%
nivel_actividad_fisica,0,0.0%
condicion_fumador,0,0.0%
mantiene_ha_alta,0,0.0%


In [11]:
friesgo_cod.shape

(4461, 16)

In [63]:
# Codifico las columnas
le = LabelEncoder()
for columnas in variables_categoricas:
    friesgo_cod[columnas] = le.fit_transform(friesgo_cod[columnas])

In [64]:
friesgo_cod.mantiene_ha_alta.unique()

array([1, 0, 2], dtype=int64)

In [51]:
friesgo_cod.mantiene_ha_alta.unique()

array([False, True, nan], dtype=object)

In [52]:
calcular_nulos_y_porcentaje(friesgo_cod)

,Nulos,Porcentaje
id,0,0.0%
cod_provincia,0,0.0%
tamanio_aglomerado,0,0.0%
sexo,0,0.0%
edad,0,0.0%
condicion_actividad,0,0.0%
ansiedad_depresion,0,0.0%
nivel_actividad_fisica,0,0.0%
condicion_fumador,0,0.0%
mantiene_ha_alta,2235,7.65%


In [53]:
friesgo_cod = friesgo_cod.dropna()

In [54]:
calcular_nulos_y_porcentaje(friesgo_cod)

,Nulos,Porcentaje
id,0,0.0%
cod_provincia,0,0.0%
tamanio_aglomerado,0,0.0%
sexo,0,0.0%
edad,0,0.0%
condicion_actividad,0,0.0%
ansiedad_depresion,0,0.0%
nivel_actividad_fisica,0,0.0%
condicion_fumador,0,0.0%
mantiene_ha_alta,0,0.0%


In [55]:
friesgo_cod.shape

(25742, 16)

**Por ultimo, se realiza la division del modelo en train-test**

In [56]:
# Division del dataset en train-test en este caso el tamaño de la muestra sera del 30%
y = friesgo_cod['mantiene_ha_alta']
X = friesgo_cod.drop(columns=['mantiene_ha_alta'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Analisis modelos

## Modelo 1: Random Forest

In [35]:
# Como realice el preprocesado en la primera seccion, voy a crear un pipeline solo con el clasificador
pipeline = Pipeline(steps=[
    ('classifier', RandomForestClassifier(random_state=42))
])

In [36]:
# Creare tambien un diccionario con parametros para Gridsearch, que me ayudara a buscar los mejores parametros para mi modelo de modo mas rapido/automatizado
parametros = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

In [40]:
# Creo mi GridSearchCV
grid_search = GridSearchCV(pipeline, parametros, scoring='roc_auc',return_train_score = True)

In [43]:
friesgo_cod.mantiene_ha_alta.dtype

dtype('O')

In [41]:
# Ajustar GridSearchCV
grid_search.fit(X_train, y_train)

ValueError: 
All the 405 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
405 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Biank\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Biank\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Biank\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\pipeline.py", line 476, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "c:\Users\Biank\AppData\Local\Pro

In [ ]:
# Mejor modelo encontrado por GridSearchCV
best_model = grid_search.best_estimator_

In [ ]:
# Evaluar modelo en el conjunto de prueba
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))

## Modelo 2

## Modelo 3